In [1]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
ds = load_dataset("Hemanth-thunder/en_ta")
print(ds)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

corpus.bcn.train 166k.csv:   0%|          | 0.00/99.2M [00:00<?, ?B/s]

en-ta train 118k.csv:   0%|          | 0.00/75.8M [00:00<?, ?B/s]

corpus.bcn.dev 1k.csv:   0%|          | 0.00/589k [00:00<?, ?B/s]

corpus.bcn.test 2k.csv:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/285630 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'en', 'ta'],
        num_rows: 285630
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'en', 'ta'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'en', 'ta'],
        num_rows: 2000
    })
})


In [3]:
def transform_example(example, idx):
    return {
        "id": str(idx),  # Generate ID as a string using the index
        "translation": {
            "en": example["en"],
            "ta": example["ta"],
        },
    }

In [4]:

transformed_ds = ds.map(transform_example, with_indices=True)
transformed_ds = transformed_ds.shuffle(seed=42)  # Optional: add a seed for reproducibility
transformed_ds["train"] = transformed_ds["train"].select(range(len(transformed_ds["train"]) // 4))
transformed_ds.save_to_disk("transformed_en_ta_dataset")
print(transformed_ds["train"])


Map:   0%|          | 0/285630 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/71407 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['Unnamed: 0', 'en', 'ta', 'id', 'translation'],
    num_rows: 71407
})


In [5]:
print(transformed_ds["train"][0:3]['translation'])

[{'en': 'A recognition that you have acquired new knowledge, new skills that equip you for the world of work as well as for further learning.\n', 'ta': 'பணியாற்றும் உலகிற்குத் தேவையான திறன்களைப் பெற்றிருக்கிறீர்கள், புதிய அறிவு பெற்றிருக்கிறீர்கள் என்பதற்கான அங்கீகாரம் இது.\n'}, {'en': 'Though he be fruitful among his brothers, an east wind shall come, the wind of the LORD shall come up from the wilderness, and his spring shall become dry, and his fountain shall be dried up: he shall spoil the treasure of all pleasant vessels.\n', 'ta': 'இவன் சகோதரருக்குள்ளே ஜனம்பெருத்தவனாயிருந்தாலும், கர்த்தருடைய காற்றாகிய கீழ்க்காற்று வனாந்தரத்திலிருந்து எழும்பிவரும்; அதனால் அவனுடைய ஊற்றுகள் வற்றிப்போகும்; அவனுடைய துரவு சுவறிப்போகும்; அது இச்சிக்கப்படத்தக்க சகல பதார்த்தங்களுள்ள உத்தம பதார்த்தங்களையும் வாரிக்கொண்டு போகும்.\n'}, {'en': 'Recently we have made an attempt to have joint exercise for disaster management between countries on the lines of joint military exercise involving different countries 

In [6]:
#we need tokenizer - goal is to split this sentence into many words - BPE Tokenizer,wordlevel,subwordlevel,wordpart tokenizer - we will use word level tokenizer
# tokenizer builds teh vocabulary , we also make use of spatial tokens for padding,<sos>,<eos>

In [7]:
!pip install tokenizers

In [8]:
import torch
import torch.nn as nn
from pathlib import Path
from typing import Any
from torch.utils.data import Dataset,DataLoader , random_split
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

In [9]:
def get_all_sentences(ds,lang):
  for item in ds:
    yield item['translation'][lang]

In [10]:
def get_or_build_tokenizer(config,ds,lang):
  tokenizer_path = Path(config['tokenizer_file'].format(lang))
  if not Path.exists(tokenizer_path):
    tokenizer = Tokenizer(WordLevel(unk_token='[UNK]'))#if unknown word is seen that is not there in the vocabulary then makes it unknown token
    tokenizer.pre_tokenizer = Whitespace()
    trainer = WordLevelTrainer(special_tokens=["[UNK]","[PAD]","[SOS]","[EOS]"],min_frequency=2)
    tokenizer.train_from_iterator(get_all_sentences(ds,lang),trainer=trainer)
    tokenizer.save(str(tokenizer_path))
  else:
    tokenizer = Tokenizer.from_file(str(tokenizer_path))
  return tokenizer

In [11]:
def get_ds(config):
  config['lang_src'] = 'en'
  config['lang_tgt'] = 'ta'
  ds_raw = transformed_ds["train"]
  tokenizer_src = get_or_build_tokenizer(config,ds_raw,"en")
  tokenizer_tgt = get_or_build_tokenizer(config,ds_raw,"ta")
  train_ds_size = int(0.9 * len(ds_raw))
  val_ds_size = len(ds_raw) - train_ds_size
  train_ds_raw, val_ds_raw = random_split(ds_raw,[train_ds_size,val_ds_size])
  train_ds = BilingualDataset(train_ds_raw,tokenizer_src,tokenizer_tgt,"en","ta",config['seq_len'])
  val_ds = BilingualDataset(val_ds_raw,tokenizer_src,tokenizer_tgt,"en","ta",config['seq_len'])


  max_len_src= 0
  max_len_tgt = 0
  for item in ds_raw:
    src_ids = tokenizer_src.encode(item['translation']['en']).ids
    tgt_ids = tokenizer_tgt.encode(item['translation']['ta']).ids
    max_len_src = max(max_len_src,len(src_ids))
    max_len_tgt = max(max_len_tgt,len(tgt_ids))
  print(f"Max length of source sentence: {max_len_src}")
  print(f"Max length of target sentence: {max_len_tgt}")
  train_dataloader = DataLoader(train_ds,batch_size=config['batch_size'],shuffle=True)
  val_dataloader = DataLoader(val_ds,batch_size=1,shuffle=True)#want to process each sentence one by one
  return train_dataloader,val_dataloader,tokenizer_src,tokenizer_tgt

In [12]:
class BilingualDataset(Dataset):#dataset that stores the tensors
  def __init__(self,ds,tokenizer_src,tokenizer_tgt,src_lang,tgt_lang,seq_len)->None:
    super().__init__()
    self.ds = ds
    self.tokenizer_src = tokenizer_src
    self.tokenizer_tgt = tokenizer_tgt
    self.src_lang = src_lang
    self.tgt_lang = tgt_lang
    self.seq_len = seq_len
    self.sos_token = torch.tensor([tokenizer_src.token_to_id('[SOS]')], dtype=torch.int64)
    self.eos_token = torch.tensor([tokenizer_src.token_to_id('[EOS]')], dtype=torch.int64)
    self.pad_token = torch.tensor([tokenizer_src.token_to_id('[PAD]')], dtype=torch.int64)


  def __len__(self):
    return len(self.ds)
  def __getitem__(self,index: Any) -> Any:
    src_target_pair = self.ds[index]
    src_text = src_target_pair['translation'][self.src_lang]
    tgt_text = src_target_pair['translation'][self.tgt_lang]
    enc_input_tokens = self.tokenizer_src.encode(src_text).ids
    dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids#ids gives hte index of original words
    #note always the sequence length and given text won't match add padding
    enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2 # [SOS]and[EOS]
    dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1 # [SOS]

    if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
      raise ValueError("Sentence is too long")
    encoder_input = torch.cat(
        [
            self.sos_token,
            torch.tensor(enc_input_tokens,dtype=torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token]*enc_num_padding_tokens,dtype=torch.int64),
        ])
    decoder_input = torch.cat(
        [
            self.sos_token,
            torch.tensor(dec_input_tokens,dtype=torch.int64),
            torch.tensor([self.pad_token]*dec_num_padding_tokens,dtype=torch.int64)
        ])
    label = torch.cat(
        [
        torch.tensor(dec_input_tokens,dtype=torch.int64),
        self.eos_token,
        torch.tensor([self.pad_token]*dec_num_padding_tokens,dtype=torch.int64)
    ])
    assert encoder_input.size(0) == self.seq_len
    assert decoder_input.size(0) == self.seq_len
    assert label.size(0) == self.seq_len
    return {
        "encoder_input":encoder_input,
        "decoder_input":decoder_input,
        "encoder_mask":(encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),#we don't want pad_tokens to be considered (1,1,seq_len)
        "decoder_mask":(decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int() & causal_mask(decoder_input.size(0)),#(1,seq_len) & (1,Seq_len,seq_len)
        "label":label,
        "src_text":src_text,
        "tgt_text":tgt_text
    }

def causal_mask(size):
  mask = torch.triu(torch.ones((1,size,size)),diagonal=1).type(torch.int) #triu ->gives every value above teh diagonal
  return mask == 0



#this is fully based on interest , pls don't suffer use Translate() lib from google

### MODEL


In [13]:
import torch
import torch.nn as nn
import math
#512 is th d_model that i am using

In [14]:
class InputEmbeddings(nn.Module):

    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # (batch, seq_len) --> (batch, seq_len, d_model)
        return self.embedding(x) * math.sqrt(self.d_model)

In [15]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        pe = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)

In [16]:
class LayerNormalization(nn.Module):

    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        mean = x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        std = x.std(dim = -1, keepdim = True) # (batch, seq_len, 1)
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

In [17]:
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

In [18]:
class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)

        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
        # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        return self.w_o(x)


In [19]:
class ResidualConnection(nn.Module):

        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)

        def forward(self, x, sublayer):
            return x + self.dropout(sublayer(self.norm(x)))

In [20]:
class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)



In [21]:
class DecoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

In [22]:
class ProjectionLayer(nn.Module):

    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return self.proj(x)

In [23]:
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # (batch, seq_len, d_model)
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # (batch, seq_len, d_model)
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def project(self, x):
        # (batch, seq_len, vocab_size)
        return self.projection_layer(x)

In [24]:
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

### Training Loop

In [25]:
def get_model(config,vocab_src_len,vocab_tgt_len):
    model = build_transformer(vocab_src_len,vocab_tgt_len,config['seq_len'],config['seq_len'],d_model=config['d_model'])
    return model

In [32]:
def get_config():
  return{
      "batch_size":8,
      "num_epochs":5,
      "lr":10**-3,
      "seq_len":300,
      "d_model":512,
      "lang_src":"en",
      "lang_tgt":"ta",
      "model_folder":"weights",
      "model_basename":"tmodel_",
      "preload": None,
      "tokenizer_file":"tokenizer_{0}.json",
      "experiment_name":"runs/tmodel"
  }
def get_weights_file_path(config,epoch:str):
    model_folder = config['model_folder']
    model_basename = config['model_basename']
    model_filename = f"{model_basename}{epoch}.pt"
    return str(Path('.')/model_folder/model_filename)

In [33]:
import torch.utils.tensorboard
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm


In [34]:
!pip install torchmetrics

In [35]:
import torchmetrics
def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')
    encoder_output = model.encode(source, source_mask)
    decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
    while True:
        if decoder_input.size(1) == max_len:
            break
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)
        prob = model.project(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat(
            [decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1
        )

        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0)


def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_step, writer, num_examples=2):
    model.eval()
    count = 0
    source_texts = []
    expected = []
    predicted = []
    try:
        with os.popen('stty size', 'r') as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        console_width = 80

    with torch.no_grad():
        for batch in validation_ds:
            count += 1
            encoder_input = batch["encoder_input"].to(device) # (b, seq_len)
            encoder_mask = batch["encoder_mask"].to(device) # (b, 1, 1, seq_len)
            assert encoder_input.size(
                0) == 1, "Batch size must be 1 for validation"

            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

            source_text = batch["src_text"][0]
            target_text = batch["tgt_text"][0]
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

            source_texts.append(source_text)
            expected.append(target_text)
            predicted.append(model_out_text)

            # Print the source, target and model output
            print_msg('-'*console_width)
            print_msg(f"{f'SOURCE: ':>12}{source_text}")
            print_msg(f"{f'TARGET: ':>12}{target_text}")
            print_msg(f"{f'PREDICTED: ':>12}{model_out_text}")

            if count == num_examples:
                print_msg('-'*console_width)
                break

    if writer:
        metric = torchmetrics.CharErrorRate()
        cer = metric(predicted, expected)
        writer.add_scalar('validation cer', cer, global_step)
        writer.flush()
        metric = torchmetrics.WordErrorRate()
        wer = metric(predicted, expected)
        writer.add_scalar('validation wer', wer, global_step)
        writer.flush()
        metric = torchmetrics.BLEUScore()
        bleu = metric(predicted, expected)
        writer.add_scalar('validation BLEU', bleu, global_step)
        writer.flush()


In [36]:
def train_model(config):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"Using device: {device}")
  Path(config['model_folder']).mkdir(parents=True,exist_ok=True)
  train_dataloader,val_dataloader,tokenizer_src,tokenizer_tgt = get_ds(config)
  model = get_model(config,tokenizer_src.get_vocab_size(),tokenizer_tgt.get_vocab_size()).to(device)
  #tensorboard
  writer = SummaryWriter(config['experiment_name'])
  optimizer = torch.optim.Adam(model.parameters(),lr=config['lr'],eps=1e-9)
  #resume model state in case of crash
  initial_epoch = 0
  global_step = 0
  if config['preload']:
    model_filename = get_weights_file_path(config,config['preload'])
    print(f"Preloading model {model_filename}")
    state = torch.load(model_filename)
    initial_epoch = state['epoch'] + 1
    optimizer.load_state_dict(state['optimizer_state_dict'])
    global_step = state['global_step']

  loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'),label_smoothing=0.1).to(device)
  for epoch in range(initial_epoch,config['num_epochs']):
    torch.cuda.empty_cache()
    model.train()
    batch_iterator = tqdm(train_dataloader,desc=f"Processing Epoch {epoch:02d}")

    for batch in batch_iterator:
      #print(batch.shape)
      encoder_input = batch['encoder_input'].to(device) #(Batch , seq_len)
      decoder_input = batch['decoder_input'].to(device) #(Batch , seq_len)
      encoder_mask = batch['encoder_mask'].to(device) #(Batch , 1 , 1 , seq_len)
      decoder_mask = batch['decoder_mask'].to(device) #(Batch , 1 , seq_len , seq_len)

      encoder_output = model.encode(encoder_input,encoder_mask)
      decoder_output = model.decode(encoder_output,encoder_mask,decoder_input,decoder_mask)
      proj_output = model.project(decoder_output)
      label = batch['label'].to(device)
      loss = loss_fn(proj_output.view(-1,tokenizer_tgt.get_vocab_size()),label.view(-1))
      batch_iterator.set_postfix({"loss":f"{loss.item():6.3f}"})
      writer.add_scalar('train_loss',loss.item(),global_step)
      writer.flush()
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      global_step += 1
    run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: batch_iterator.write(msg), global_step, writer)
    model_filename = get_weights_file_path(config,f"{epoch:02d}")
    torch.save({
        'epoch':epoch,
        'model_state_dict':model.state_dict(),
        'optimizer_state_dict':optimizer.state_dict(),
        'global_step':global_step
    },model_filename)

In [ ]:
config = get_config()
train_model(config)

Using device: cuda
Max length of source sentence: 280
Max length of target sentence: 272


Processing Epoch 00: 100%|██████████| 8034/8034 [46:50<00:00,  2.86it/s, loss=7.694]


--------------------------------------------------------------------------------
    SOURCE: It lacks basic physical and social infrastructure, including roads, health care and education.

    TARGET: அங்கு வீதிகள், சுகாதார பராமரிப்பு மற்றும் கல்வி உட்பட அடிப்படையான சரீர மற்றும் சமூக உட்கட்டமைப்பு பற்றாக்குறை நிலவுகிறது.

 PREDICTED: 
--------------------------------------------------------------------------------
    SOURCE: Among all my films released so far, this is the mega hit.

    TARGET: இதுவரை ரிலிஸான எனது படங்களிலேயே இந்த படம்தான் மெகா ஹிட்.

 PREDICTED: 
--------------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `BLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `BLEUScore` from `torchmetrics.text` instead.
  _future_warning(
Processing Epoch 01: 100%|██████████| 8034/8034 [46:51<00:00,  2.86it/s, loss=6.686]


--------------------------------------------------------------------------------
    SOURCE: When we met him, he was busy with the background music and dubbing work.

    TARGET: பின்னணி இசை, டப்பிங் வேலையில் பரபரப்பாக இருந்தவரை சந்தித்த வேளையில்....

 PREDICTED: 
--------------------------------------------------------------------------------
    SOURCE: There's no thrill in running alone.

    TARGET: தனி குதிரையாக ஓடுவதில் த்ரில் இல்லை.

 PREDICTED: 
--------------------------------------------------------------------------------


Processing Epoch 02: 100%|██████████| 8034/8034 [46:49<00:00,  2.86it/s, loss=7.840]


--------------------------------------------------------------------------------
    SOURCE: There is a conspiracy of her prophets in the middle thereof, like a roaring lion ravening the prey; they have devoured souls; they have taken the treasure and precious things; they have made her many widows in the middle thereof.

    TARGET: அதிலுள்ள தீர்க்கதரிசிகள் அதின் நடுவில் கட்டுப்பாடு பண்ணுகிறார்கள்; கெர்ச்சிக்கிற சிங்கம் இரை கவ்வுகிறதுபோல, ஆத்துமாக்களை அவர்கள் பட்சிக்கிறார்கள்; திரவியத்தையும் விலையுயர்ந்த பொருள்களையும் வாங்கிக்கொள்ளுகிறார்கள்; அதின் நடுவில் அநேகரை விதவைகளாக்குகிறார்கள்.

 PREDICTED: 
--------------------------------------------------------------------------------
    SOURCE: Ministry of Development of North-East Region North East Special Infrastructure Development Scheme In December 2017, Government of India approved North East Special Infrastructure Development Scheme (NESIDS) with an outlay of Rs.1600.00 crore for the period of 2017-18 to 2019-20.

    TARGET: வடகிழக

Processing Epoch 03: 100%|██████████| 8034/8034 [46:49<00:00,  2.86it/s, loss=7.047]


--------------------------------------------------------------------------------
    SOURCE: Meera Vasudevan has come into serials even while acting in films.

    TARGET: மீராவாசுதேவன் படங்களில் நடித்துக்கொண்டே சீரியலுக்கும் வந்துவிட்டார்.

 PREDICTED: 
--------------------------------------------------------------------------------
    SOURCE: The Union Minister of Human Resource Development, Shri Prakash Javadekar has said that there is an urgent need to rationalize the school curriculum because the objective of education is to prepare a good human being from the system.

    TARGET: பள்ளிகளின் பாடத்திட்டத்தை நியாயப்படுத்தி மாற்றி அமைப்பதற்கு அவசர அவசியம் உள்ளது என்று தெரிவித்த மத்திய மனிதவள மேம்பாட்டுத்துறை அமைச்சர் திரு பிரகாஷ் ஜவடேகர், கல்வியின் நோக்கம் கல்வி முறையின் மூலம் நல்ல மனிதர்களை தயார் செய்வது ஆகும் என்று கூறினார்.

 PREDICTED: 
--------------------------------------------------------------------------------


Processing Epoch 04:  23%|██▎       | 1830/8034 [10:40<36:06,  2.86it/s, loss=7.122]

### inference

In [ ]:
import torch
import sys
def translate(sentence: str):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    config = get_config()
    tokenizer_src = Tokenizer.from_file(str(Path(config['tokenizer_file'].format(config['lang_src']))))
    tokenizer_tgt = Tokenizer.from_file(str(Path(config['tokenizer_file'].format(config['lang_tgt']))))
    model = build_transformer(tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size(), config["seq_len"], config['seq_len'], d_model=config['d_model']).to(device)
    model_filename = latest_weights_file_path(config)
    state = torch.load(model_filename)
    model.load_state_dict(state['model_state_dict'])
    label = ""
    if type(sentence) == int or sentence.isdigit():
        id = int(sentence)
        ds = load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='all')
        ds = BilingualDataset(ds, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
        sentence = ds[id]['src_text']
        label = ds[id]["tgt_text"]
    seq_len = config['seq_len']
    model.eval()
    with torch.no_grad():
        source = tokenizer_src.encode(sentence)
        source = torch.cat([
            torch.tensor([tokenizer_src.token_to_id('[SOS]')], dtype=torch.int64),
            torch.tensor(source.ids, dtype=torch.int64),
            torch.tensor([tokenizer_src.token_to_id('[EOS]')], dtype=torch.int64),
            torch.tensor([tokenizer_src.token_to_id('[PAD]')] * (seq_len - len(source.ids) - 2), dtype=torch.int64)
        ], dim=0).to(device)
        source_mask = (source != tokenizer_src.token_to_id('[PAD]')).unsqueeze(0).unsqueeze(0).int().to(device)
        encoder_output = model.encode(source, source_mask)
        decoder_input = torch.empty(1, 1).fill_(tokenizer_tgt.token_to_id('[SOS]')).type_as(source).to(device)
        if label != "": print(f"{f'ID: ':>12}{id}")
        print(f"{f'SOURCE: ':>12}{sentence}")
        if label != "": print(f"{f'TARGET: ':>12}{label}")
        print(f"{f'PREDICTED: ':>12}", end='')
        while decoder_input.size(1) < seq_len:
            decoder_mask = torch.triu(torch.ones((1, decoder_input.size(1), decoder_input.size(1))), diagonal=1).type(torch.int).type_as(source_mask).to(device)
            out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)
            prob = model.project(out[:, -1])
            _, next_word = torch.max(prob, dim=1)
            decoder_input = torch.cat([decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1)
            print(f"{tokenizer_tgt.decode([next_word.item()])}", end=' ')
            if next_word == tokenizer_tgt.token_to_id('[EOS]'):
                break
    return tokenizer_tgt.decode(decoder_input[0].tolist())



In [ ]:
translate(sys.argv[1] if len(sys.argv) > 1 else "I am not a very good a student.")